In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240724'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/1.group.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Group No'] = input_['Group No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        payload = {'jsn': {'tab': 'extras',
                           'groupname': input_.loc[a, 'Group'],
                           'nodetype': 'groupname'}}

        data = {'func': 'navnode_fetch',
                'payload': json.dumps(payload),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['jsn']['tab'] == 'extras' and json_['jsn']['groupname'] == input_.loc[a, 'Group'] and json_['jsn']['nodetype'] == 'groupname' and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_json = html.xpath('//input[contains(@id, "jsn[__GIP__")]/@value')

            # = = = = = = = = = = = = = = =

            list_subgroup = [json.loads(json_)['subgroupname'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            if not list_subgroup:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Group No': input_.loc[a, 'Group No'],
                                    'Group': input_.loc[a, 'Group'],
                                    'Subgroup No': [i+1 for i in range(len(list_subgroup))],
                                    'Subgroup': list_subgroup})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                                     'Group': input_.loc[a, 'Group']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Group No']}.{input_.loc[a, 'Group']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Group No', 'Subgroup No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/2.subgroup-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Group No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/subgroup_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：26

[状态：ok，尝试次数：1] - 6.Drivetrain
[剩余数量：6] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 3.Body & Lamp Assembly
[剩余数量：5] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 17.Hoses/Lines & Clamps
[剩余数量：4] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 2.Belt Drive
[剩余数量：3] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 4.Brake & Wheel Hub
[剩余数量：2] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 20.Literature
[剩余数量：1] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 16.Heat & Air Conditioning
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 13.Fuel & Air
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 7.Electrical
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 9.Electrical-Connector
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 1.Apparel & Gifts
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 19.Interior
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 14.Garage Equipment
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 5.Cooling System
[剩余数量：0] - [当前时间：16:31:33]

[状态：ok，尝试次数：1] - 22.Suspension
[剩余数量：0] - [当前时间：16:31:34]

[状态：ok，尝试次数：1] - 18.Ignition
[剩余数量：0] - [当前时间：16:31: